## Imports

In [157]:
# General imports
import time
import json
import random
import os

# Intent recognition imports
import torch
torch.manual_seed(42)
from transformers import BertTokenizer, BertForSequenceClassification

# ChatGPT imports
%load_ext autoreload
%autoreload 2
from gpt import GPT
from settings import Settings
from intent_model import IntentModel

# Chat demo
import gradio as gr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [158]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
random.seed(SEED)
# numpy.random.seed(SEED)
from transformers import set_seed
set_seed(SEED)

# Load data 

In [159]:
# Load the data from JSON file
with open("intent_recognition_v2.json", "r") as f:
    data = json.load(f)

validation_data = []
train_data = []
for datum in data:
    texts = datum["train_questions"]
    label = datum["intent"]
    validation = datum["test_questions"]
    for text in texts:
        train_data.append((text, label))
    for text in validation:
        validation_data.append((text, label))

# Concatenate train and validation data for generating label_map
all_data = train_data + validation_data

# Define the mapping between top-level labels and integers
# Sort the labels before enumerating
label_map = {label: i for i, label in enumerate(sorted(set([data[1] for data in all_data])))}

# Convert the training data labels to integers using the label_map
train_labels = torch.tensor([label_map[data[1]] for data in train_data])
validation_labels = torch.tensor([label_map[data[1]] for data in validation_data])



# Intent recognition model

In [160]:
intent_model = IntentModel(train_data, validation_data, label_map, train_labels, validation_labels)


In [161]:
# intent_model = IntentModel(train_data, validation_data, train_label_map, train_labels, validation_labels).test_best_model()

In [162]:
print(intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")[0], ": minimum age")
print(intent_model.get_intent("Could you summarize the main provisions of the legal statement?")[0], ": legal_statement_information")
print(intent_model.get_intent("Are you allowed to transfer ownership of my account or subscription to another individual?")[0], ": transfer_legal_statement")

minimum_age : minimum age
legal_statement_information : legal_statement_information
transfer_legal_statement : transfer_legal_statement


In [163]:
import pandas as pd

# test the intent model
predicted_intent, confidence_scores = intent_model.get_intent("Can you provide details on the rules pertaining to the age limit for using the company's services?")

# Retrieve the intent label and its corresponding confidence score
highest_confidence_intent = predicted_intent

print("Highest confidence intent:", highest_confidence_intent)
print("Highest confidence score:", confidence_scores)
print(len(confidence_scores))
print(len(intent_model.label_map.keys()))

pd.DataFrame({
    'keys': intent_model.label_map.keys(),
    'confidences': confidence_scores
})

Highest confidence intent: minimum_age
Highest confidence score: [0.004220775794237852, 0.0017763165524229407, 0.00266229547560215, 0.004789701197296381, 0.0033615082502365112, 0.0015738876536488533, 0.0029093995690345764, 0.0011391140287742019, 0.001752939191646874, 0.0013188798911869526, 0.0009560642065480351, 0.9530506730079651, 0.0023339232429862022, 0.002781984629109502, 0.0026278633158653975, 0.005179507192224264, 0.0011820412473753095, 0.001031978870742023, 0.00336332805454731, 0.0019878819584846497]
20
20


,keys,confidences
0,access_personal_data,0.004221
1,cancellation_legal_statement,0.001776
2,contact_privacy_concerns,0.002662
3,cookie_policy,0.004790
4,data_breaches,0.003362
5,data_collection,0.001574
6,data_sharing_opt_out,0.002909
7,data_storage,0.001139
8,delete_personal_information,0.001753
9,hello_response,0.001319


## ChatGPT model

In [164]:
# GPT model here
gpt_model = GPT()

privacy_level = 1 # set to 2 for maximum privacy
minimum_confidence_score = 0.4

# gpt_model = False

# Test ChatGPT model
# gpt_model.answer_question(question='What is the most important thing I need to know about your privacy statement?')

# Load Settings class

In [165]:
settings_model = Settings()

# Test the settings model
# settings_model.update_user_setting("1", "profile_published", False)
settings_model.set_user_calling_card_visibility(user_id = "0", individual_user_id = "2", profile_card_component = "profile_card_component_3_visible", component_state = True)



## Get response using intent

In [166]:
def get_object_by_intent(intent):
    for object in data:
        if object['intent'] == intent:
            return object
    return None

# Set up gradio chat for demo

In [167]:
theme = gr.themes.Soft(
    primary_hue="orange",
    secondary_hue="orange",
).set(
    body_text_color_dark='*neutral_800',
    background_fill_primary_dark='*neutral_50',
    background_fill_secondary_dark='*neutral_50',
    border_color_accent_dark='*primary_300',
    border_color_primary_dark='*neutral_200',
    color_accent_soft_dark='*primary_50',
    link_text_color_dark='*secondary_600',
    link_text_color_active_dark='*secondary_600',
    link_text_color_hover_dark='*secondary_700',
    link_text_color_visited_dark='*secondary_500',
    block_background_fill='*neutral_100',
    block_background_fill_dark='*neutral_100',
    block_label_background_fill='*primary_400',
    block_label_background_fill_dark='*primary_400',
    block_label_text_color='*neutral_50',
    block_label_text_color_dark='*neutral_50',
    block_title_text_color='*neutral_50',
    block_title_text_color_dark='*neutral_50',
    checkbox_background_color_dark='*background_fill_primary',
    checkbox_background_color_selected='*primary_500',
    checkbox_background_color_selected_dark='*primary_500',
    checkbox_border_color_dark='*neutral_100',
    checkbox_border_color_focus='*primary_300',
    checkbox_border_color_focus_dark='*primary_300',
    checkbox_border_color_hover_dark='*neutral_300',
    checkbox_border_color_selected='*primary_500',
    checkbox_border_color_selected_dark='*primary_500',
    checkbox_border_width_dark='1px',
    checkbox_label_background_fill_selected_dark='*primary_500',
    checkbox_label_text_color_selected_dark='white',
    error_background_fill_dark='#fee2e2',
    error_border_color_dark='#fecaca',
    input_background_fill_dark='white',
    input_background_fill_focus_dark='*secondary_500',
    input_border_color_dark='*neutral_50',
    input_border_color_focus_dark='*secondary_300',
    input_placeholder_color_dark='*neutral_400',
    slider_color_dark='*primary_500',
    stat_background_fill_dark='*primary_300',
    table_border_color_dark='*neutral_300',
    table_even_background_fill_dark='white',
    table_odd_background_fill_dark='*neutral_50',
    button_primary_background_fill_dark='*primary_500',
    button_primary_background_fill_hover_dark='*primary_400',
    button_primary_border_color_dark='*primary_200',
    button_secondary_background_fill_dark='white',
    button_secondary_background_fill_hover_dark='*neutral_100',
    button_secondary_border_color_dark='*neutral_200',
    button_secondary_text_color_dark='*neutral_800'
)

class ChatbotLogic:
    def __init__(self, gpt_model, intent_model, minimum_confidence_score, privacy_level):
        self.gpt_model = gpt_model
        self.intent_model = intent_model
        self.minimum_confidence_score = minimum_confidence_score
        self.privacy_level = privacy_level
        self.history = []
        self.dialogue = {'active': False, 'counter': 0}

        # Add welcome message
        self.add_bot_response("Hello, I am the iYYU privacy bot. I am here to help you with your questions about the company.")
        
    def user(self, user_message, history):
        return "", self.add_user_message(user_message)

    def add_user_message(self, user_message):
        self.history.append([user_message, None])
        return self.history

    def bot(self, history):
        user_message = self.history[-1][0]
        bot_response = self.determine_bot_response(user_message)
        self.history = self.add_bot_response(bot_response)

        # if not bot_response.startswith("ChatGPT - "):
        #     time.sleep(1)
        return self.history

    def add_bot_response(self, bot_response):
        self.history.append([None, bot_response])
        return self.history
        

    def determine_bot_response(self, user_message):
        '''
        Condition numbers match up with the condition numbers in the chart
        
        '''
        # Condition 1: Is the dialogue active?
        if (self.dialogue['active'] == True):
            '''
                Use an array of questions to have a dialogue with the user and use the users answers
            '''
            response = self.add_bot_response('(Test): Starting dialogue')

            # Condition 2: Are there dialogue questions left?
            dialogue_questions = ['test', 'test2', 'test3']
            if (dialogue.counter < len(dialogue_questions)):
                response = self.add_bot_response('(Test): More dialogue questions')
            else:
                response = self.add_bot_response('(Test): Ran out of dialogue questions, computing answer then awaiting user input')

        else:
            print('2')
            intent_name, confidence_score = self.intent_model.get_intent(question=user_message)
            confidence_score = max(confidence_score)
            intent_data = get_object_by_intent(intent_name)
            use_gpt = intent_data['use_gpt']
            generate_context = True

            # Condition 3: Is the confidence score for the intent high enough to use it?
            if confidence_score < self.minimum_confidence_score:
                # Condition 4: Does the user have a high or low privacy level?
                if self.privacy_level >= 1: 
                    response = self.add_bot_response('(Test): Low confidence, high security answer')
                else:
                    response = self.add_bot_response('(Test): Low confidence, low security answer (use chatgpt)')
            else:
                # Condition 5: Is there intent that requires a specific action?
                if intent_name == 'initalize_user_settings':
                    # Logic to ask user questions to set the settings here
                    response = self.add_bot_response('(Test): Specific intent, initialize settings')
                elif intent_name == 'change_setting':
                    # NER keywoard recognition implementation here
                    response = self.add_bot_response('(Test): Specific intent, change specific setting')
                else:
                    # Condition 6: Is the intent using GPT?
                    if (not use_gpt):
                        # Predefined response
                        response = self.add_bot_response('(Test): Send the predefined response (settings respones)')
                    else:
                        if self.privacy_level >= 1: 
                            response = self.add_bot_response('(Test): Send predefined question to chatgpt')
                        else:
                            response = self.add_bot_response('(Test): Send users questions directly to chatgpt')


        # Finally return the response
        return response

def initialize_chat():
    logic = ChatbotLogic(gpt_model, intent_model, minimum_confidence_score, privacy_level)
    return logic.history

with gr.Blocks(theme=theme, css="chat/chat.css") as demo:
    gr.Image("https://iyyu.com/_nuxt/img/navbar_logoW@2x.79eba99.png", interactive=False, tool="image", show_label=False, elem_classes="logo").style(width=200)
    chatbot = gr.Chatbot(initialize_chat)
    msg = gr.Textbox()
    submit = gr.Button("Submit")

    msg.submit(logic.user, [msg, chatbot], [msg, chatbot], queue=False).then(
        logic.bot, chatbot, chatbot
    )
    submit.click(logic.user, [msg, chatbot], [msg, chatbot], queue=False).then(
        logic.bot, chatbot, chatbot
    )

demo.launch()




Running on local URL:  http://127.0.0.1:7908

To create a public link, set `share=True` in `launch()`.


3


Traceback (most recent call last):
  File "c:\Program Files\Python38\lib\site-packages\gradio\routes.py", line 395, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Program Files\Python38\lib\site-packages\gradio\blocks.py", line 1193, in process_api
    result = await self.call_function(
  File "c:\Program Files\Python38\lib\site-packages\gradio\blocks.py", line 916, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Program Files\Python38\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Program Files\Python38\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Program Files\Python38\lib\site-packages\anyio\_backends\_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "C:\Users\Frank\AppData\Local\Temp\ipykernel_148652\4006651907.py", line 80, in bot
    bot_res